In [1]:

from belly import *


In [2]:
import os

os.environ['BELLY_SIZE'] = '200'
os.environ['BELLY_LOCATION'] = 'gs://spain-tweets/datalake'

In [ ]:
main()

In [ ]:
from schemas import *

In [ ]:
new_schema = create_schema()

In [ ]:
import pickle
from gcsfs import GCSFileSystem

new_schema = create_schema()
fs = GCSFileSystem(project='trollhunters')

with fs.open('gs://spain-tweets/schemas/tweet-clean.pickle', 'wb') as f:
    pickle.dump(new_schema, f)